In [ ]:
!pip install langchain[all] chromadb
!pip install layoutparser[layoutmodels,tesseract]
!pip install unstructured
!pip install unstructured[local-inference]
!pip install --ignore-installed Pillow==9.0.0

In [ ]:
!pip install google-search-results

In [ ]:
!pip install configparser

In [ ]:
import configparser
api_config = configparser.ConfigParser()
api_config.read_file(open('apidata.config'))

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = api_config["OPENAI"]["KEY"]
os.environ['SERPAPI_API_KEY']=api_config["OPENAI"]["SERPAPI_KEY"]
os.environ['SERPER_API_KEY']=api_config["OPENAI"]["SERPER_KEY"]

In [ ]:
from langchain.llms import OpenAI

langModel = OpenAI(temperature=0.9)
text = "Provide me top 5 birthday gift ideas for a 18 year old?"
print(langModel(text))



1. Electronic Accessories: Music Players, Earphones, Smart Watches etc
2. Subscription to a Video Streaming Service
3. A New Laptop or Smartphone
4. Sport Equipment or Clothing
5. A Weekend Vacation


In [ ]:
from langchain import PromptTemplate

one_input_prompt = PromptTemplate(input_variables=["topic"], 
                                  template="Provide me {topic} for a 18 year old.")
one_input_prompt.format(topic="3 books suggestion")

'Provide me 3 books suggestion for a 18 year old.'

In [ ]:
multiple_input_prompt = PromptTemplate(
    input_variables=["topic", "age"], 
    template="Provide me top 3 {topic} for a {age} year old."
)
multiple_input_prompt.format(topic='Cartoons',age='10')

'Provide me top 3 Cartoons for a 10 year old.'

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

from langchain.chains import LLMChain

topicChain = LLMChain(llm=langModel, prompt=multiple_input_prompt)
topicChain.run(topic='Tourist Spots', age = 21)

'\n\n1. Great Barrier Reef, Australia\n2. Grand Canyon, USA\n3. Giza Necropolis, Egypt'

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

fact_llm = OpenAI(temperature=0)

tool_chain = load_tools(["serpapi", "llm-math"], llm=fact_llm)

agent_decider = initialize_agent(tool_chain, 
                                 fact_llm, 
                                 agent="zero-shot-react-description", 
                                 verbose=True)


agent_decider.run("When was Earth born? And how many days it has been since its birth?")



> Entering new AgentExecutor chain...
 I need to find out when Earth was born and how many days have passed since then.
Action: Search
Action Input: "When was Earth born?"
Observation: about 4.5 billion years ago
Thought: I need to calculate how many days have passed since then
Action: Calculator
Action Input: 4.5 billion years * 365 days
Observation: Answer: 16422500000000 days
Thought: I now know the final answer
Final Answer: Earth was born about 4.5 billion years ago and it has been 16422500000000 days since then.

> Finished chain.


'Earth was born about 4.5 billion years ago and it has been 16422500000000 days since then.'

In [ ]:
from langchain import ConversationChain
convo_chain = ConversationChain(llm=fact_llm, verbose=True)
convo_chain.predict(input='Convert the 16422500000000 in to billion')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Convert the 16422500000000 in to billion
AI:

> Finished chain.


' 16422500000000 is equal to 1642.25 billion.'

In [ ]:
convo_chain.predict(input='The number I gave earlier was in days, convert it into hours')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Convert the 16422500000000 in to billion
AI:  16422500000000 is equal to 1642.25 billion.
Human: The number I gave earlier was in days, convert it into hours
AI:

> Finished chain.


' 16422500000000 days is equal to 3.945e+14 hours.'

In [ ]:
from langchain import SQLDatabase, SQLDatabaseChain

In [ ]:
lite_db = SQLDatabase.from_uri("sqlite:///chinook.db")

In [ ]:
db_chain = SQLDatabaseChain(llm=fact_llm, database=lite_db, verbose=True)
db_chain.run("How many tables are in the database?")



> Entering new SQLDatabaseChain chain...
How many tables are in the database? 
SQLQuery: SELECT COUNT(*) FROM sqlite_master WHERE type='table';
SQLResult: [(4,)]
Answer: There are 4 tables in the database.
> Finished chain.


' There are 4 tables in the database.'

In [ ]:
db_chain.run("List the table names in the database")



> Entering new SQLDatabaseChain chain...
List the table names in the database 
SQLQuery: SELECT name FROM sqlite_master WHERE type='table';
SQLResult: [('Album',), ('Artist',), ('Genre',), ('MediaType',)]
Answer: The table names in the database are Album, Artist, Genre, and MediaType.
> Finished chain.


' The table names in the database are Album, Artist, Genre, and MediaType.'

In [ ]:
db_chain.run("How many Unique Artists are there?")



> Entering new SQLDatabaseChain chain...
How many Unique Artists are there? 
SQLQuery: SELECT COUNT(DISTINCT ArtistId) FROM Artist;
SQLResult: [(275,)]
Answer: There are 275 Unique Artists.
> Finished chain.


' There are 275 Unique Artists.'